### rOMERO-gateway Overview
![Overview](romero_overview.png)

### Load rOMERO-gateway

In [ ]:
options(warn=-1)
library(romero.gateway)

### Connect to the server

In [ ]:
server <- OMEROServer(host = 'outreach.openmicroscopy.org', username='username', password='password', port= as.integer(4064))
server <- connect(server)

## Read Data from OMERO

### OMERO dataset and Annotation details
Please fill in the datasetId for the dataset named 'CellProfiler images'

In [ ]:
datasetId <- 1076
fileName <- 'Cells.csv'

### Find the dataset and get the attachment

In [ ]:
omeDataset <- loadObject(server, 'DatasetData', datasetId)
print(paste('Dataset ID:',omeDataset@dataobject$getId()))

# Find the attached csv file
fileAnnotation <- getAnnotations(server, 'DatasetData', getOMEROID(omeDataset), nameFilter = fileName)
annotationFileID = as.integer(fileAnnotation$FileID)
annotationFileID <- annotationFileID[[length(annotationFileID)]]
print(paste('Annotation ID:', annotationFileID))

# Get Image Names
images <- getImages(omeDataset)
image_names <- c()
for (image in images){
    image_names <- c(image_names, image@dataobject$getName())
}

### Load annotation as a R-Dataframe and compare cell areas between the two sets of images

In [ ]:
df <- loadCSV(server, annotationFileID)
dfLength <- subset(df, select=c("ImageNumber", "AreaShape_Area"))
boxplot(dfLength$AreaShape_Area~dfLength$ImageNumber)

In [ ]:
Areas = subset(dfLength, dfLength$ImageNumber == image_names[[1]])$AreaShape_Area
hist(Areas, main = paste("Frequency Histogram of Cellular Areas:",image_names[[1]]))

In [ ]:
Areas = subset(dfLength, dfLength$ImageNumber == image_names[[2]])$AreaShape_Area
hist(Areas, main = paste("Frequency Histogram of Cellular Areas:",image_names[[2]]))

## Save Results back to OMERO

### Attach the plot to an OMERO dataset 

In [ ]:
# Attach the plot to the dataset
tmpfile <- "/tmp/boxplot.png"
png(tmpfile)
boxplot(dfLength$AreaShape_Area~dfLength$ImageNumber)
dev.off()
omero_attach_plot <- attachFile(omeDataset, tmpfile)

print(paste("OMERO File Annotation ID:", omero_attach_plot@dataobject$getId()))

### Attach the t-test output to the dataset

In [ ]:
Areas_2 <- subset(dfLength, dfLength$ImageNumber == image_names[[1]] | dfLength$ImageNumber == image_names[[2]])

t.test(Areas_2$AreaShape_Area~Areas_2$ImageNumber)
tmpfile <- "/tmp/t_test_summary.txt"
cat("Here are my results:\n", file=tmpfile)
capture.output(t.test(Areas_2$AreaShape_Area~Areas_2$ImageNumber), file=tmpfile, append=TRUE)
omero_attach_file <- attachFile(omeDataset, tmpfile)

print(paste("OMERO File Annotation ID:", omero_attach_file@dataobject$getId()))

### Disconnect OMERO session

In [ ]:
server_disconnect <- disconnect(server)